In [1]:
import pandas as pd
import numpy as np
import json
from datetime import date
import urllib

# Создать файлы с данными по elo для пар героев ко дню проведения матча с мнимальным количеством игр=10

In [52]:
columns = ['hero', 'kills', 'deaths', 'assists', 'kda', 'avgKal', 'gpm', 'xpm', 'lastHits', 'denies', 'level',
           'heroDamage', 'towerDamage', 'heroHealing', 'goldSpent']
# создать файлы с elo пар героев по каждой дате (с 2017 года используется только патч 7,00+)
year = 2018
for month in range (2, 3):
    for day in range (1, 14):
         # создать правильную ссылку для поиска по всем героям 
        url_heroes =('http://www.datdota.com/api/heroes/head-to-head-elo?tier=1&valve-event=does-not-matter&threshold=10' +
                     '&patch=7.09&patch=7.08&patch=7.07&patch=7.06&patch=7.05&patch=7.04&patch=7.03&patch=7.02' +
                     '&patch=7.01&patch=7.00&winner=either&after=01%2F01%2F2011' +
                     '&before={}%2F{}%2F{}&duration=0%3B200&'.format(day, month, year) +
                     'duration-value-from=0&duration-value-to=200') 
        # выгрзить json с предыдущей ссылки
        sauce = urllib.request.Request(url_heroes, headers={'User-agent': 'Mozilla/5.0'} )
        dat = json.loads(urllib.request.urlopen(sauce).read().decode()).get('data')
        df = pd.DataFrame(dat)
        
        # запись фала на диск
        df.to_csv('../tabel with heroes DATDOTA (elo for hero vs enemy)/{}-{}-{}.csv'.format(year, month, day))

# Подгрузка мейн ДФ с id матчей

In [3]:
# загрузить главный ДФ
main = pd.read_csv('../tabel/MAIN TABLE PREMIUM.csv', index_col=0)
# main = main[:4]
# main.loc[:, 'radiant_win':'dire_H5']
main = main[main['match_id'] == 3274564960]
main

,match_id,start_time,radiant_team_id,radiant_score,dire_team_id,dire_score,radiant_win,radiant_H1,radiant_H2,radiant_H3,...,radiant_P2,radiant_P3,radiant_P4,radiant_P5,dire_P1,dire_P2,dire_P3,dire_P4,dire_P5,league_name
1007,3274564960,1498454706,3659536.0,13,3326875.0,20,False,111.0,47.0,99.0,...,164532005.0,121833213.0,152545459.0,127077041.0,100471531.0,21289303.0,89603649.0,84772440.0,85417034.0,The International 2017


# Создание таблицы с данными elo для каждого героя, относительно его противников для работы

In [9]:
# добавить в main_df суммарный показатель elo для героя vs всх врагов
def elo_heroes_vs_enemies(index, columns_heroes, columns_enemies, df_elo_herVsEne, main_df):
    # две переменные для записи суммарного elo каждой команды
    for her in columns_heroes:
        hero_elo = 0
        # вытащить id героя
        id_heroe = main[her][index]
        print ('-------------------------------------------------------')
        print (her, ' - ', id_heroe)
        # суммировать elo героя против героев противника
        for her_enemy in columns_enemies:
            id_heroe_enemy = main[her_enemy][index]
            try:
                elo =  df_elo_herVsEne['shift'][df_elo_herVsEne['hero'] == 
                                              id_heroe][df_elo_herVsEne['againstHero'] == id_heroe_enemy].item()
            except:
                elo=0
            hero_elo += elo
            print ('id врага - ', id_heroe_enemy)
            print('elo врага - ', elo)
        # записать в основной ДФ elo по каждому герою
#         print('elo героя суммарное - ', hero_elo)
        main_df.loc[index, her + 'elo_vs_enemies'] =  hero_elo 

In [10]:
# создать основной массив, где будут записаны суммарное elo каждого егроя относительно всех героев противника
df_head_to_head_elo_heroes = pd.DataFrame()#main['match_id']

# создать список названий колонок  героев radiant
all_her_rad = main.loc[:,'radiant_H1':'radiant_H5'].columns
# создать список названий колонок  героев radiant
all_her_dir = main.loc[:,'dire_H1':'dire_H5'].columns

# ВЫТАЩИТЬ ЕЛО ГЕРОЕВ ЗА ВСЕ ПАТЧИ И С ДАТОЙ ИЮНЬ 2018, ТАК КАК ТАК БОЛЕЕ ПРАВЕЛЬНЕЕ СМОТРЕТЬ КОНТРПИКИ
# создать правильную ссылку для посика героев по команде radiant
url_heroes_team = ('http://www.datdota.com/api/heroes/head-to-head-elo?tier=1&valve-event=does-not-matter&threshold=10&patch=7.09&patch=7.08&patch=7.07&patch=7.06&patch=7.05&patch=7.04&patch=7.03&patch=7.02&patch=7.01&patch=7.00&patch=6.88&patch=6.87&patch=6.86&patch=6.85&patch=6.84&patch=6.83&patch=6.82&patch=6.81&patch=6.80&patch=6.79&patch=6.78&patch=6.77&patch=6.76&patch=6.75&patch=6.74&winner=either&after=01%2F01%2F2011&before=05%2F06%2F2018&duration=0%3B200&duration-value-from=0&duration-value-to=200')
# выгрзить json с предыдущей ссылки и создать ДФ
sauce = urllib.request.Request(url_heroes_team, headers={'User-agent': 'Mozilla/5.0'} )
dat = json.loads(urllib.request.urlopen(sauce).read().decode()).get('data')
df_elo_herVsEne = pd.DataFrame(dat)

for index in main.index:
    # достать дату матча
    date_match = date.fromtimestamp(main['start_time'][index])
    
#     # Создать ДФ с ело для пар героев
#     df_elo_herVsEne = pd.read_csv('../tabel with heroes DATDOTA (elo for hero vs enemy)/{}-{}-{}.csv'.format(date_match.year, date_match.month, date_match.day-1), index_col=0)
    

    
    # добавить в мейн таблицу данные сначала по ело героям рединт против дире, а затем наоборот
    elo_heroes_vs_enemies(index, all_her_rad, all_her_dir, df_elo_herVsEne, df_head_to_head_elo_heroes )
    elo_heroes_vs_enemies(index, all_her_dir, all_her_rad, df_elo_herVsEne, df_head_to_head_elo_heroes )
    
df_head_to_head_elo_heroes['match_id'] = main['match_id']    

-------------------------------------------------------
radiant_H1  -  111.0
id врага -  28.0
elo врага -  -0.25191617873086924
id врага -  25.0
elo врага -  -3.334703384245572
id врага -  75.0
elo врага -  -3.3369282404797174
id врага -  65.0
elo врага -  -0.8756506009871377
id врага -  6.0
elo врага -  -7.545113068301073
-------------------------------------------------------
radiant_H2  -  47.0
id врага -  28.0
elo врага -  -7.50750821913813
id врага -  25.0
elo врага -  5.332878539495614
id врага -  75.0
elo врага -  3.39706077003479
id врага -  65.0
elo врага -  -6.039483013200279
id врага -  6.0
elo врага -  -2.714869782068599
-------------------------------------------------------
radiant_H3  -  99.0
id врага -  28.0
elo врага -  -3.084959133417381
id врага -  25.0
elo врага -  2.971836025000672
id врага -  75.0
elo врага -  -2.705398342574399
id врага -  65.0
elo врага -  0.49008536108017126
id врага -  6.0
elo врага -  -0.37233535278914087
-------------------------------------

In [6]:
df_head_to_head_elo_heroes#.to_csv('../tabel/table from Datdota/tabel elo heroes in match vs enemies (head-to_head, contrPicks).csv')

,radiant_H1elo_vs_enemies,radiant_H2elo_vs_enemies,radiant_H3elo_vs_enemies,radiant_H4elo_vs_enemies,radiant_H5elo_vs_enemies,dire_H1elo_vs_enemies,dire_H2elo_vs_enemies,dire_H3elo_vs_enemies,dire_H4elo_vs_enemies,dire_H5elo_vs_enemies,match_id
1007,-15.344311,-7.531922,-2.700771,16.159748,2.75975,-2.03825,-14.791996,-6.54423,4.154835,9.947005,3274564960


# Создание elo  для каждого героя относительно его врагов для каждой команды отдельно и добавление его в main df

In [32]:
# вытащить таблицу с elo героев для команды 
def get_df_elo_herVsEne_teams(radiant_or_dire):
    # id команды из данных по матчу
    id_team = int(main[radiant_or_dire + '_team_id'][index])

    # создать правильную ссылку для посика героев по команде radiant
    url_heroes_team = ('http://www.datdota.com/api/heroes/head-to-head-elo?' + 
                       'teams={}&tier=1&valve-event=does-not-matter&threshold=1'.format(id_team) + 
                       '&patch=7.09&patch=7.08&patch=7.07&patch=7.06&patch=7.05&patch=7.04' + 
                       '&patch=7.03&patch=7.02&patch=7.01&patch=7.00&patch=6.88&winner=either&' + 
                       'after=01%2F01%2F2011&before={}%2F{}%2F{}'.format(date_match.day-1, date_match.month, date_match.year) + 
                       '&duration=0%3B200&duration-value-from=0&duration-value-to=200')
    print (url_heroes_team)
    # выгрзить json с предыдущей ссылки и создать ДФ
    sauce = urllib.request.Request(url_heroes_team, headers={'User-agent': 'Mozilla/5.0'} )
    dat = json.loads(urllib.request.urlopen(sauce).read().decode()).get('data')
    df = pd.DataFrame(dat)
    return df

In [33]:
df_head_to_head_elo_heroes_teams = pd.DataFrame()#main['match_id']

# создать список названий колонок  героев radiant
all_her_rad = main.loc[:,'radiant_H1':'radiant_H5'].columns
# создать список названий колонок  героев radiant
all_her_dir = main.loc[:,'dire_H1':'dire_H5'].columns

for index in main.index:
    # достать дату матча
    date_match = date.fromtimestamp(main['start_time'][index])
    # вытащить таблицу с elo героев для команды radiant
    
    # Создать ДФ с ело для пар героев
    df_elo_herVsEne_rad = get_df_elo_herVsEne_teams('radiant')
    df_elo_herVsEne_dire = get_df_elo_herVsEne_teams('dire')
    elo_heroes_vs_enemies(index, all_her_rad, all_her_dir, df_elo_herVsEne_rad, df_head_to_head_elo_heroes_teams )
    elo_heroes_vs_enemies(index, all_her_dir, all_her_rad, df_elo_herVsEne_dire, df_head_to_head_elo_heroes_teams )
    
df_head_to_head_elo_heroes_teams['match_id'] = main['match_id']   

http://www.datdota.com/api/heroes/head-to-head-elo?teams=1883502&tier=1&valve-event=does-not-matter&threshold=1&patch=7.09&patch=7.08&patch=7.07&patch=7.06&patch=7.05&patch=7.04&patch=7.03&patch=7.02&patch=7.01&patch=7.00&patch=6.88&winner=either&after=01%2F01%2F2011&before=5%2F1%2F2017&duration=0%3B200&duration-value-from=0&duration-value-to=200
http://www.datdota.com/api/heroes/head-to-head-elo?teams=350190&tier=1&valve-event=does-not-matter&threshold=1&patch=7.09&patch=7.08&patch=7.07&patch=7.06&patch=7.05&patch=7.04&patch=7.03&patch=7.02&patch=7.01&patch=7.00&patch=6.88&winner=either&after=01%2F01%2F2011&before=5%2F1%2F2017&duration=0%3B200&duration-value-from=0&duration-value-to=200
86.0
id врага -  54.0
elo врага -  0
id врага -  46.0
elo врага -  0
id врага -  62.0
elo врага -  9.531489693434205
id врага -  25.0
elo врага -  0
id врага -  28.0
elo врага -  0
elo героя суммарное -  9.531489693434205
8.0
id врага -  54.0
elo врага -  0
id врага -  46.0
elo врага -  -38.36994331282

In [34]:
df_head_to_head_elo_heroes_teams

,radiant_H1elo_vs_enemies,radiant_H2elo_vs_enemies,radiant_H3elo_vs_enemies,radiant_H4elo_vs_enemies,radiant_H5elo_vs_enemies,dire_H1elo_vs_enemies,dire_H2elo_vs_enemies,dire_H3elo_vs_enemies,dire_H4elo_vs_enemies,dire_H5elo_vs_enemies,match_id
0,9.53149,-25.103871,11.562659,-42.307475,-67.215331,0.996375,-97.087518,47.410273,0.0,13.016238,2897064589


In [69]:
df_head_to_head_elo_heroes

,radiant_H1elo_vs_enemies,radiant_H2elo_vs_enemies,radiant_H3elo_vs_enemies,radiant_H4elo_vs_enemies,radiant_H5elo_vs_enemies,dire_H1elo_vs_enemies,dire_H2elo_vs_enemies,dire_H3elo_vs_enemies,dire_H4elo_vs_enemies,dire_H5elo_vs_enemies,match_id
0,-9.357245,-5.159937,3.743775,-2.616665,-7.090932,-10.996970,7.213008,13.888981,-16.892703,1.957910,2897064589
1,-12.088595,-6.632346,0.149019,2.745266,0.416305,13.302127,0.383357,-10.322423,0.646065,-1.467602,2897074199
2,-3.537030,-17.376757,-10.341601,-19.145237,-13.489237,9.709519,2.073755,10.142552,4.298129,20.581079,2897155244
3,3.742056,6.414117,6.586189,7.825294,-12.280674,6.018056,-5.359727,-12.109310,-5.111870,-17.867914,2897190211
4,-2.306375,21.288938,-4.462887,-2.513072,-4.125175,-2.424278,-4.400896,-0.429505,-13.513951,-2.460016,2897294037
5,1.161036,7.756609,-13.493298,-6.698015,20.534968,-11.772721,-8.396368,21.614975,-14.286369,-11.802413,2897415426
6,-4.094305,-8.674045,-3.326673,-11.930558,-7.816857,-5.074840,-0.670342,-4.766884,12.986489,16.326079,2897508853
7,-7.364036,0.262581,-11.445124,-7.956097,-10.660047,-13.245455,24.625319,-0.467988,5.086649,-6.176507,2897669822
8,10.116658,-5.211082,1.204601,14.497008,-11.823908,0.841078,-0.952344,-13.994744,-10.636639,3.607780,2897839994
9,6.861648,-2.391959,-30.367217,-9.338557,3.094026,-14.676884,5.858441,12.022960,14.063930,4.998167,2898057696
